# Ollama 101 - A set of basic ollama functions from the ollama pypi docs
All functions are referenced from the PyPi ollama package built by Jeffrey Morgan: https://pypi.org/project/ollama/

The following notebook is a set of ollama tools and part of an ollama guide made by Leo Borcherding. 
This notebook is part of the following ollama tutorial https://github.com/Leoleojames1/ollamaStarterKit

In [1]:
# Install Ollama
!pip install ollama
# Install ipywidgets
!pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


# Ollama pull/run

The following commands will allow you to pull any ollama model, feel free to explore ollama's vast set of models to choose from, here are some of my custom system prompts: https://ollama.com/borch

ollama run phi3

&

ollama pull phi3

# Ollama show & list

*ollama show, shows the currently loaded model's modelfile metadata*
ollama show --modelfile llama3.2:3b

*ollama list shows the available models*
ollama list

In [2]:
# Import the Ollama library
import ollama
import json
from datetime import datetime

# Show information about a specific model
model_info = ollama.show('llama3.2:3b')

# Convert to a dictionary for easier formatting
model_dict = {
    "model": "llama3.2:3b",
    "modified_at": str(model_info.modified_at),
    "family": model_info.details.family,
    "parameter_size": model_info.details.parameter_size,
    "quantization": model_info.details.quantization_level,
    "parameters": model_info.parameters.split('\n')
}

# Pretty print the model information
print(json.dumps(model_dict, indent=2))

# List all available models
models = ollama.list()

# Print a simplified table of models
print("\n{:<40} {:<10} {:<10} {:<10}".format("MODEL", "FAMILY", "SIZE", "QUANT"))
print("-" * 75)

for model in models.models:
    size_gb = f"{model.size / 1_000_000_000:.2f} GB"
    print("{:<40} {:<10} {:<10} {:<10}".format(
        model.model[:38], 
        model.details.family[:8], 
        size_gb, 
        model.details.quantization_level[:8]
    ))


{
  "model": "llama3.2:3b",
  "modified_at": "2024-09-25 17:58:55.324918-05:00",
  "family": "llama",
  "parameter_size": "3.2B",
  "quantization": "Q4_K_M",
  "parameters": [
    "stop                           \"<|start_header_id|>\"",
    "stop                           \"<|end_header_id|>\"",
    "stop                           \"<|eot_id|>\""
  ]
}

MODEL                                    FAMILY     SIZE       QUANT     
---------------------------------------------------------------------------
hf.co/unsloth/gemma-3-4b-it-GGUF:Q4_K_   gemma3     3.34 GB    unknown   
hf.co/Borcherding/Llama-3.2-3B-Finetom   llama      2.02 GB    unknown   
hf.co/Borcherding/OARC_Commander_v001_   llama      6.43 GB    unknown   
hf.co/Borcherding/OARC_Commander_v001_   llama      3.42 GB    unknown   
dolphin3:latest                          llama      4.92 GB    Q4_K_M    
marco-o1:latest                          qwen2      4.68 GB    Q4_K_M    
qwen2.5-coder:latest                     qwen2   

# Chat Response

The ChatResponse method can be used for direct back and forth messaging with your ollama model, but will not provide access to prompt streaming. For Streaming follow along to the next section.

In [3]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='llama3.2:3b', messages=[
  {
    'role': 'user',
    'content': 'Who are you? And what were you trained on?',
  },
])
print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)

I'm an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI." 

I was trained on a combined text dataset consisting of books and articles from the internet. This training allows me to generate human-like responses to a wide range of questions and topics.
I'm an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI." 

I was trained on a combined text dataset consisting of books and articles from the internet. This training allows me to generate human-like responses to a wide range of questions and topics.


# Streaming Response



Here's a nice interactive chat interface for your Ollama models that works directly in the notebook. This interface includes:

1. A dropdown to select the ollama model
2. A text input for messages
3. A scrollable chat history display
4. A reset button to start fresh conversations
5. Streaming responses that appear character by character

To use it, simply run this cell and you'll get a fully functional chat interface:

In [ ]:
import ollama
import time
from IPython.display import clear_output

class SimpleOllamaChat:
    def __init__(self, model="llama3.2:3b"):
        """Initialize the chat interface with a specified model."""
        self.model = model
        self.messages = []
        self.running = True
        print(f"🤖 Chat initialized with model: {self.model}")
        print("Type 'exit' to end the chat, 'clear' to reset the conversation, or 'change model:[name]' to switch models.")
        
    def display_models(self):
        """Display available models in a clean format."""
        try:
            models_list = ollama.list()
            print("\nAvailable models:")
            print("-" * 60)
            for model in models_list.models[:10]:  # Show only first 10 models to avoid clutter
                print(f"• {model.model}")
            if len(models_list.models) > 10:
                print(f"... and {len(models_list.models) - 10} more models")
            print("-" * 60)
        except Exception as e:
            print(f"⚠️ Error fetching models: {e}")
    
    def start_chat(self):
        """Start the interactive chat loop."""
        while self.running:
            # Get user input
            user_message = input("\n👤 You: ")
            
            # Process commands
            if user_message.lower() == 'exit':
                print("👋 Ending chat session.")
                self.running = False
                break
                
            elif user_message.lower() == 'clear':
                self.messages = []
                clear_output(wait=True)
                print(f"🤖 Chat reset with model: {self.model}")
                print("Type 'exit' to end the chat, 'clear' to reset the conversation, or 'change model:[name]' to switch models.")
                continue
                
            elif user_message.lower().startswith('change model:'):
                new_model = user_message[len('change model:'):].strip()
                if new_model:
                    self.model = new_model
                    print(f"🔄 Model changed to {self.model}")
                    continue
                else:
                    self.display_models()
                    continue
                    
            elif user_message.lower() == 'models':
                self.display_models()
                continue
                
            # Skip empty messages
            if not user_message.strip():
                continue
                
            # Add user message to history
            self.messages.append({'role': 'user', 'content': user_message})
            
            # Get AI response with streaming
            print("\n🤖 AI: ", end='')
            
            try:
                full_response = ""
                stream = ollama.chat(
                    model=self.model,
                    messages=self.messages,
                    stream=True,
                )
                
                for chunk in stream:
                    if 'message' in chunk and 'content' in chunk['message']:
                        text_chunk = chunk['message']['content']
                        full_response += text_chunk
                        print(text_chunk, end='', flush=True)
                
                # Add AI response to history
                self.messages.append({'role': 'assistant', 'content': full_response})
                print()  # Add newline after response
                
            except Exception as e:
                print(f"\n⚠️ Error: {str(e)}")
                
                # If model not found, suggest listing models
                if "model not found" in str(e).lower():
                    print(f"Model '{self.model}' not found. Type 'models' to see available models.")
                    
                    # Reset to a likely available model
                    default_models = ["llama3.2:3b", "llama3.2", "llama3", "phi3"]
                    for model in default_models:
                        try:
                            # Try to pull model info to check if it exists
                            ollama.show(model)
                            self.model = model
                            print(f"Switched to model: {self.model}")
                            break
                        except:
                            continue

# Create and start the chat interface
chat = SimpleOllamaChat()
chat.start_chat()

🤖 Chat initialized with model: llama3.2:3b
Type 'exit' to end the chat, 'clear' to reset the conversation, or 'change model:[name]' to switch models.

🤖 AI: Hello! How can I assist you today?

🤖 AI: I'm just a language model, so I don't have emotions or feelings like humans do. But I'm functioning properly and ready to help with any questions or topics you'd like to discuss!

How about you? How's your day going so far?

🤖 AI: Power series solutions for differential equations are a fundamental topic in mathematics and physics. Here's an overview of the method, along with some LaTeX code examples.

**Method**

Assume you have a second-order linear homogeneous differential equation of the form:

$$y'' + p(x)y' + q(x)y = 0$$

where $p(x)$ and $q(x)$ are continuous functions on an interval $[a, b]$. We want to find a power series solution of the form:

$$y(x) = \sum_{n=0}^{\infty} a_n x^n$$

Substituting this expression into the differential equation, we get:

$$(\sum_{n=2}^{\infty} n(n-1)a